# Generating Poetic Texts with Recurrent Neural Networks

(https://www.geeksforgeeks.org/introduction-to-recurrent-neural-network/)

Reccurent Neural Networks (RNNs) are a class of neural networks that is
    powerful for modeling sequence data such as time series or natural
    language. What distinguishes RNNs from other neural networks is that
    they have a "memory" that captures information about what has been
    calculated so far. In theory, RNNs can make use of information in
    arbitrarily long sequences, but in practice they are limited to looking
    back only a few steps. This is because the RNNs are trained by
    "unrolling" them into very deep neural networks, which becomes
    computationally expensive.

LSTM is a type of Recurrent Neural Network (RNN) architecture that is very
    effective in learning long-term dependencies. It has been successfully
    applied to a variety of sequence modeling tasks, such as language
    modeling, speech recognition, and text compression

RMSprop is an unpublished, adaptive learning rate method proposed by Geoff
    Hinton in Lecture 6e of his Coursera Class.
Root Mean Square Propagation is an adaptive learning rate method, which
    is a very effective technique for training neural networks. It is
    especially effective for problems with sparse gradients, which is very
    common in Natural Language Processing tasks.

In [15]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

filepath = tf.keras.utils.get_file('shakespeare.txt',
                                   'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# .lower() is used to minimize the size of the data set
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

# select some part of the text
text = text[300_000:800_000]
# set is to select only unique characters
characters = sorted(set(text))

# create a dictionary to map characters to indices
# enumerate() is used to iterate over a list and keep track of the index
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

# print(char_to_index) # examples : { ..., '?': 12, 'a': 13, ... }
# print(index_to_char) # examples : { ..., 12: '?', 13: 'a', ... }

# how many features(chars) we are going to use to predict the next character
SEQ_LENGTH = 40
# how many chars we are going to shift to create a new sequence
# ????? Need more explanation for step_zise = 3:
# ex: "how are you" -> "ow are you " -> "w are you h" -> " are you ho" -> "are you how"
STEP_SIZE = 3

sentences = [] # the input sentence , ex: "how are yo"
next_characters = [] # the output character , ex: "u"

# range(0, len(text) - SEQ_LENGTH, STEP_SIZE) ex: range(0, 100, 3) -> [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, ...]
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    # sentences are unfinished sequences of length SEQ_LENGTH
    sentences.append(text[i: i + SEQ_LENGTH])
    # and next characters are the SEQ_LENGTH-th character in the text
    # which completes the sentence(the correct prediction)
    next_characters.append(text[i + SEQ_LENGTH])

'''
nr_of_sentences x SEQ_LENGTH x nr_of_characters (3D array)
1 dimension for all the possible sentences
1 dimension for all the individual characters in a sentence
1 dimension for all the possible characters in the text
so if the character is present in the sentence
x[sentence_index][character_index][character] = 1 otherwise 0
'''
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool) # input
# which character comes after a given sentence
y = np.zeros((len(sentences), len(characters)), dtype=bool) # output

for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_characters[i]]] = 1

model = Sequential()

# LSTM will remember the last 128 characters
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
# numberof neurons = number of characters
model.add(Dense(len(characters)))
model.add(Activation('softmax')) # softmax is used to normalize the output

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

# batch_size = how many sentences we are going to use for each training step
model.fit(x, y, batch_size=256, epochs=4)

model.save('text_generator.model')

Epoch 1/4
651/651 [==============================] - 134s 201ms/step - loss: 2.1499
Epoch 2/4
651/651 [==============================] - 56s 85ms/step - loss: 1.7217
Epoch 3/4
651/651 [==============================] - 55s 85ms/step - loss: 1.5934
Epoch 4/4
651/651 [==============================] - 57s 87ms/step - loss: 1.5247
INFO:tensorflow:Assets written to: text_generator.model\assets


INFO:tensorflow:Assets written to: text_generator.model\assets


In [17]:
model_load = tf.keras.models.load_model('text_generator.model')

# The following function is used to generate the next character
# based on the probability distribution of the output
# temperature is used to control the randomness of the output
# the higher the temperature the more random the output
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t , character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1

        predictions = model_load.predict(x, verbose=0)[0]
        # print(predictions)
        next_index = sample(predictions, temperature)
        # print(next_index)
        next_character = index_to_char[next_index]

        generated += next_character
        # exclude the first character and add the next character at the end
        sentence = sentence[1:] + next_character
    return generated

print('-----------------0.2-----------------')
print(generate_text(300, 0.2))
print('-----------------0.4-----------------')
print(generate_text(300, 0.4))
print('-----------------0.6-----------------') # the best
print(generate_text(300, 0.6))
print('-----------------0.8-----------------') # or this one
print(generate_text(300, 0.8))
print('-----------------1-----------------')
print(generate_text(300, 1))

-----------------0.2-----------------
unish'd, that have minded you
of what you the brack and the father thee
and the hath the with the brother the brother stand thee the stands.

king richard ii:
the waster the heart to the stands of the stands.

king richard ii:
the true a stands and the the hath the brother,
and the state have so the stands the brother both
the brack the w
-----------------0.4-----------------
was forged, with my rapier's point.

duke of york:
the break in the morn to the hate and the traitor.

first mayill:
my lord, by the properate and the country.

gloucester:
the wonder's love to the fortune the with thee
and the hath the batter so stands to the brother,
and the heart to the speak the speak of shall for thee the winder
and 
-----------------0.6-----------------
s,
no more my king, for he dishonours mersbled in thee
is my slawn boying weeps in the great and the crastiless.

king richard ii:
i will thou killorn thee to the offerel no thee,
that have and in the gro